In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [3]:
%cd gdrive/MyDrive

/content/gdrive/MyDrive


In [5]:
model_path= '/content/gdrive/MyDrive/NTI/Automated Video Surveillance/face_verification/datasetV2/best_vgg_data_new_correct.h5'
vggface_path = '/content/gdrive/MyDrive/NTI/Automated Video Surveillance/face_verification/vgg_face_weights.h5'

In [6]:
import pickle
import numpy
import tensorflow
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau , EarlyStopping
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

sns.set(style='white', context='notebook', palette='deep')

In [7]:
import tensorflow as tf

def my_activation(x):
    return tf.keras.activations.tanh(x)

In [8]:
!pip install deepface

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.4 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116932 sha256=caf170470caf5698e722f534fd006dd722759218350b71d76f83c752d82b2df4
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [9]:
from tensorflow.keras.models import Model, load_model
from keras.applications import VGG16
from keras.layers import Lambda
from deepface.basemodels import VGGFace
import tensorflow as tf

def make_siamese_model():
    # Anchor image input in the network
    input_image = Input(name='input_img', shape=(224,224,3))
    
    # Validation image in the network 
    validation_image = Input(name='validation_img', shape=(224,224,3))

    vgg = VGGFace.baseModel()
    vgg.load_weights(vggface_path)
    cnt = 0
    for lyer in vgg.layers:
        if cnt > 34:
            lyer.trainable = True
            lyer.activation= tf.keras.layers.Activation('tanh')
        else:
            lyer.trainable = False
        cnt += 1
    vgg_face = Model(inputs=vgg.layers[0].input, outputs=vgg.layers[-2].output)

    x1 = vgg_face(input_image)
    x2 = vgg_face(validation_image)

    x1 = Flatten()(x1)
    x2 = Flatten()(x2)

    distances = Lambda(lambda x: tf.math.abs(x[0] - x[1]), name='L1_distance')([x1, x2])
    classifier = Dense(3, activation='softmax')(distances)
    
    siamese_model = Model(inputs=[input_image, validation_image], outputs=classifier)

    # Load the saved model with custom object scope
    with tf.keras.utils.custom_object_scope({'my_activation': my_activation}):
        saved_model_path = model_path
        loaded_model = tf.keras.models.load_model(saved_model_path)

    return loaded_model

In [10]:
import tensorflow as tf

def my_activation(x):
    return tf.keras.activations.tanh(x)

# Register the custom activation function with Keras
tf.keras.utils.get_custom_objects().update({'my_activation': my_activation})

# Define the saved model path
saved_model_path = model_path

# Load the saved model with the custom activation function
with tf.keras.utils.custom_object_scope({'Activation': tf.keras.layers.Activation,
                                          'my_activation': my_activation}):
    loaded_model = tf.keras.models.load_model(saved_model_path)

In [11]:
loaded_model

In [12]:
df = pd.read_csv('/content/gdrive/MyDrive/NTI/Automated Video Surveillance/face_verification/datasetV2/datasetv2.csv')

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df, test_size = 0.3, random_state=42)

In [14]:
X_test

,anchor,validation,lable
992,/content/gdrive/MyDrive/NTI/Automated Video Su...,/content/gdrive/MyDrive/NTI/Automated Video Su...,0
746,/content/gdrive/MyDrive/NTI/Automated Video Su...,/content/gdrive/MyDrive/NTI/Automated Video Su...,0
1066,/content/gdrive/MyDrive/NTI/Automated Video Su...,/content/gdrive/MyDrive/NTI/Automated Video Su...,0
425,/content/gdrive/MyDrive/NTI/Automated Video Su...,/content/gdrive/MyDrive/NTI/Automated Video Su...,2
490,/content/gdrive/MyDrive/NTI/Automated Video Su...,/content/gdrive/MyDrive/NTI/Automated Video Su...,2
...,...,...,...
259,/content/gdrive/MyDrive/NTI/Automated Video Su...,/content/gdrive/MyDrive/NTI/Automated Video Su...,2
204,/content/gdrive/MyDrive/NTI/Automated Video Su...,/content/gdrive/MyDrive/NTI/Automated Video Su...,1
547,/content/gdrive/MyDrive/NTI/Automated Video Su...,/content/gdrive/MyDrive/NTI/Automated Video Su...,0
649,/content/gdrive/MyDrive/NTI/Automated Video Su...,/content/gdrive/MyDrive/NTI/Automated Video Su...,0


In [17]:
!pip install visualkeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
visualkeras.layered_view(loaded_model, legend=True, draw_volume=False) 

NameError: ignored

In [ ]:
reacall

In [ ]:
i=0
for image in os.listdir('/content/gdrive/MyDrive/Automated Video Surveillance/face_verification/raw images/Dataset'):
    validation_img = os.path.join('application_data', 'verification_images', image)
    print(validation_img)
    i = i+1
    if i >= 30:
      break

In [19]:
ver_s_path = '/content/gdrive/MyDrive/NTI/Automated Video Surveillance/face_verification/raw images/verify sarah'
ver_y_path = '/content/gdrive/MyDrive/NTI/Automated Video Surveillance/face_verification/raw images/verify yasmeen'
imgn = '/content/gdrive/MyDrive/Automated Video Surveillance/face_verification/datasetV2/negative/Abdullah_0004.jpg'
imgs = '/content/gdrive/MyDrive/NTI/Automated Video Surveillance/face_verification/raw images/Dataset/-5883973069897316348_121.jpg'
imgy ='/content/gdrive/MyDrive/NTI/Automated Video Surveillance/face_verification/raw images/yasmeen mobil uploads/B612_20210704_191522_620.jpg'

In [20]:
def verify(model, detection_threshold, verification_threshold, verify_path,img_path):
    # Build results array
    results = []
    ver = verify_path
    for image in os.listdir(ver):
        input_img = preprocess(img_path)
        validation_img = preprocess(os.path.join(ver, image))
        
        # Make Predictions 
        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)
  
    
    return results

In [22]:
def preprocess(file_path):
    
    # Read in image from file path
    byte_img = tf.io.read_file(file_path)
    # Load in the image 
    img = tf.io.decode_jpeg(byte_img)
    
    # Preprocessing steps - resizing the image to be 100x100x3
    img = tf.image.resize(img, (224,224))
    # Scale image to be between 0 and 1 
    img = img / 255.0

    # Return image
    return img

In [24]:
import os
results = verify(loaded_model, 0.5, 0.5,ver_s_path, imgs)
#print(results)

1/1 [==============================] - 1s 1s/step


In [25]:
resulty = verify(loaded_model, 0.5, 0.5,ver_y_path, imgy)

1/1 [==============================] - 1s 1s/step


In [26]:
for i in range(15):
  highest_index = np.argmax(results[i], axis=1)
  print(highest_index)

[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]


In [27]:
for i in range(15):
  highest_index = np.argmax(resulty[i], axis=1)
  print(highest_index)

[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]


In [ ]:
import os
results = verify(loaded_model, 0.5, 0.5)
print(results)

1/1 [==============================] - 1s 1s/step
[array([[0.03752322, 0.24093848, 0.7215383 ]], dtype=float32), array([[5.290949e-04, 9.988869e-01, 5.840310e-04]], dtype=float32), array([[7.653819e-07, 9.999987e-01, 6.038304e-07]], dtype=float32), array([[4.2246098e-05, 9.9977976e-01, 1.7802036e-04]], dtype=float32), array([[4.9112034e-05, 9.9991155e-01, 3.9373084e-05]], dtype=float32), array([[9.1288959e-05, 9.9941993e-01, 4.8881077e-04]], dtype=float32), array([[2.0232414e-06, 9.9999690e-01, 1.0279774e-06]], dtype=float32), array([[1.673543e-06, 9.999981e-01, 2.849162e-07]], dtype=float32), array([[1.2325497e-06, 9.9999869e-01, 1.7288949e-07]], dtype=float32), array([[2.9637897e-05, 9.9994731e-01, 2.2998565e-05]], dtype=float32), array([[3.9687725e-06, 9.9999142e-01, 4.6009818e-06]], dtype=float32), array([[2.7687178e-05, 9.9994755e-01, 2.4812896e-05]], dtype=float32), array([[1.4027746e-06, 9.9999809e-01, 4.3583680e-07]], dtype=float32), array([[2.9467460e-06, 9.9999511e-01, 1.8849

In [ ]:
results


[array([[0.03752322, 0.24093848, 0.7215383 ]], dtype=float32),
 array([[5.290949e-04, 9.988869e-01, 5.840310e-04]], dtype=float32),
 array([[7.653819e-07, 9.999987e-01, 6.038304e-07]], dtype=float32),
 array([[4.2246098e-05, 9.9977976e-01, 1.7802036e-04]], dtype=float32),
 array([[4.9112034e-05, 9.9991155e-01, 3.9373084e-05]], dtype=float32),
 array([[9.1288959e-05, 9.9941993e-01, 4.8881077e-04]], dtype=float32),
 array([[2.0232414e-06, 9.9999690e-01, 1.0279774e-06]], dtype=float32),
 array([[1.673543e-06, 9.999981e-01, 2.849162e-07]], dtype=float32),
 array([[1.2325497e-06, 9.9999869e-01, 1.7288949e-07]], dtype=float32),
 array([[2.9637897e-05, 9.9994731e-01, 2.2998565e-05]], dtype=float32),
 array([[3.9687725e-06, 9.9999142e-01, 4.6009818e-06]], dtype=float32),
 array([[2.7687178e-05, 9.9994755e-01, 2.4812896e-05]], dtype=float32),
 array([[1.4027746e-06, 9.9999809e-01, 4.3583680e-07]], dtype=float32),
 array([[2.9467460e-06, 9.9999511e-01, 1.8849572e-06]], dtype=float32),
 array([[5

In [ ]:
results[0]

array([[0.03752322, 0.24093848, 0.7215383 ]], dtype=float32)

In [ ]:
other = 0
sarah = 0
yasmeen = 0
for i in range(70):
  highest_index = np.argmax(results[i], axis=1)
  print(highest_index)
  if highest_index == 0:
    other = other+1
  elif highest_index == 1:
    sarah = sarah+1
  elif highest_index==2:
    yasmeen = yasmeen +1

if sarah > yasmeen and sarah > other:
  face = 'sarah'
elif yasmeen >other and yasmeen >sarah:
  face = 'yasmeen'
elif other >sarah and other > yasmeen:
  face = 'other'

[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
